In [ ]:
import random
import time
import csv
import os
import socket
import numpy as np
from codecarbon import EmissionsTracker
import inspyred
from inspyred import ec

# Configuración del problema
INDIVIDUAL_SIZE = 1024
MAX_GENERATIONS = 100000
MAX_TIME_SECONDS = 120
RUNS_PER_CONFIG = 10

# Configuraciones a probar
POPULATION_SIZES = [2**6, 2**10, 2**14]
CROSSOVER_RATES = [0.2, 0.01, 0.8]
MUTATION_RATE = 0.1

COMPUTER_NAME = socket.gethostname()
results_file = f'resultados_inspyred_{COMPUTER_NAME}_3.csv'
codecarbon_file = f'codecarbon_inspyred_results_{COMPUTER_NAME}_3.csv'

# Clase para almacenar las estadísticas de evolución
class EvolutionStats:
    def __init__(self):
        self.initial_fitness = 0
        self.best_fitness = 0
        self.gen_best_fitness = 0
        self.current_generation = 0
        self.termination_cause = "timeout"

# Instancia global para almacenar estadísticas
stats = EvolutionStats()

def onemax_fitness(candidates, args):
    return [sum(candidate) / INDIVIDUAL_SIZE for candidate in candidates]

def generator(random, args):
    return [random.choice([0, 1]) for _ in range(args.get('individual_size', INDIVIDUAL_SIZE))]

# Definimos un terminador personalizado que combina tiempo y generaciones
def time_generation_termination(population, num_generations, num_evaluations, args):
    global stats
    
    max_time = args.get('max_time', MAX_TIME_SECONDS)
    max_generations = args.get('max_generations', MAX_GENERATIONS)
    start_time = args.get('start_time')
    
    # Actualizar generación actual
    stats.current_generation = num_generations
    
    # Verificar si hemos excedido el número máximo de generaciones
    if num_generations >= max_generations:
        stats.termination_cause = "max_generations"
        return True
    
    # Verificar si hemos excedido el tiempo máximo
    if time.time() - start_time >= max_time:
        stats.termination_cause = "timeout"
        return True
    
    return False

# Función para observar y registrar información
def stats_observer(population, num_generations, num_evaluations, args):
    global stats
    
    if not population:
        return
    
    # Obtener el mejor fitness de la población actual
    current_best = max([ind.fitness for ind in population])
    
    # Si es la primera generación, guardar como fitness inicial
    if num_generations == 0:
        stats.initial_fitness = current_best
    
    # Actualizar el mejor fitness encontrado hasta ahora
    if current_best > stats.best_fitness:
        stats.best_fitness = current_best
        stats.gen_best_fitness = num_generations

def main():
    # Crear archivo de resultados si no existe
    file_exists = os.path.isfile(results_file)
    
    with open(results_file, 'a', newline='') as csvfile:
        fieldnames = ['population_size', 'crossover_rate', 'mutation_rate', 
                      'run', 'generations', 'initial_fitness', 'best_fitness', 'fitness_variation', 'time', 'energy_consumed']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if not file_exists:
            writer.writeheader()
        
        # Probar cada combinación de parámetros
        for pop_size in POPULATION_SIZES:
            for cx_rate in CROSSOVER_RATES:
                for run in range(1, RUNS_PER_CONFIG + 1):
                    print(f"Ejecutando: Población={pop_size}, Cruce={cx_rate}, Ejecución={run}")
                    
                    # Reiniciar las estadísticas para esta ejecución
                    global stats
                    stats = EvolutionStats()
                    
                    # Configurar el algoritmo genético
                    rand = random.Random()
                    rand.seed(time.time())  # Semilla aleatoria para cada ejecución
                    
                    ea = ec.GA(rand)
                    ea.selector = inspyred.ec.selectors.tournament_selection
                    ea.variator = [inspyred.ec.variators.uniform_crossover, 
                                   inspyred.ec.variators.bit_flip_mutation]
                    ea.replacer = inspyred.ec.replacers.generational_replacement
                    
                    # Usar el terminador personalizado
                    ea.terminator = time_generation_termination
                    
                    # Añadir el observador para registrar estadísticas
                    ea.observer = stats_observer
                    
                    # Guardar la información de la ejecución para el registro de codecarbon
                    os.environ["CODECARBON_RUN_ID"] = f"onemax_p{pop_size}_cx{cx_rate}_r{run}"
                    
                    # Configurar el seguimiento de emisiones
                    tracker = EmissionsTracker(
                        project_name="OneMax_Experiment",
                        output_dir='.', 
                        output_file=codecarbon_file, 
                        save_to_file=True, 
                        log_level="critical",
                        tracking_mode="process",
                        allow_multiple_runs=False
                    )
                    
                    # Iniciar seguimiento de emisiones
                    tracker.start()
                    start_time = time.time()
                    
                    # Ejecutar el algoritmo genético con una sola llamada
                    final_pop = ea.evolve(generator=generator,
                                    evaluator=onemax_fitness,
                                    pop_size=pop_size,
                                    maximize=True,
                                    bounder=inspyred.ec.Bounder(0, 1),
                                    max_generations=MAX_GENERATIONS,
                                    start_time=start_time,
                                    max_time=MAX_TIME_SECONDS,
                                    num_selected=pop_size,
                                    individual_size=INDIVIDUAL_SIZE,
                                    crossover_rate=cx_rate,
                                    mutation_rate=MUTATION_RATE,
                                    tournament_size=2 
                                    )
                    
                    # Obtener el tiempo transcurrido
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    
                    # Detener el seguimiento de emisiones
                    emissions = tracker.stop()
                    
                    if emissions is None:  # En caso de que codecarbon no pueda calcular las emisiones
                        emissions = 0
                    
                    # Usar los datos almacenados en nuestro objeto stats
                    initial_fitness = stats.initial_fitness
                    best_fitness = stats.best_fitness
                    generations = stats.current_generation + 1  # +1 porque empezamos desde 0
                    
                    # Calcular la variación del fitness
                    fitness_variation = best_fitness - initial_fitness
                    
                    # Guardar resultados
                    writer.writerow({
                        'population_size': pop_size,
                        'crossover_rate': cx_rate,
                        'mutation_rate': MUTATION_RATE,
                        'run': run,
                        'generations': generations,
                        'initial_fitness': initial_fitness,
                        'best_fitness': best_fitness,
                        'fitness_variation': fitness_variation,
                        'time': elapsed_time,
                        'energy_consumed': emissions
                    })
                    
                    # Asegurarse de que los datos se escriban al archivo después de cada ejecución
                    csvfile.flush()
                    
                    print(f"Finalizado: Aptitud inicial={initial_fitness:.4f}, Mejor aptitud={best_fitness:.4f}, Variación={fitness_variation:.4f}")
                    print(f"Generaciones={generations}, Tiempo: {elapsed_time:.2f}s, Energía: {emissions} kWh")
                    print(f"Motivo de parada: {stats.termination_cause}")
                    print("-" * 50)

if __name__ == "__main__":
    main()
